In [ ]:
# === Notebook bootstrap: make repo root importable ===
import sys
from pathlib import Path

_cwd = Path.cwd().resolve()
for p in [_cwd, *_cwd.parents]:
    if (p / "src").exists():
        if str(p) not in sys.path:
            sys.path.insert(0, str(p))
        break

print("cwd:", _cwd)
print("sys.path[0]:", sys.path[0])


In [ ]:
#01
import pandas as pd
import numpy as np

from src.io_utils import load_pickle, save_pickle, outputs_dir
from src.calendar_utils import add_calendar_columns

OUT_DIR = outputs_dir()
DF_W_PATH = Path(OUT_DIR) / "df_w.pkl"

df_w = load_pickle(DF_W_PATH)

print("df_w shape:", df_w.shape)
display(df_w.head())


In [ ]:
#02
# t（トレンド）
df_w = df_w.sort_values("Week").reset_index(drop=True)
df_w["t"] = np.arange(len(df_w))


In [ ]:
#03
# 日曜始まりのweek_start / FY週を付与（単一ソース化）
df_w = add_calendar_columns(df_w, date_col="Week", fy_start_month=7, fy_start_day=1)

# 互換性維持：既存で week_of_year を参照している箇所があるので iso_week を入れておく
# ※ 53週があれば 53 のまま保持（潰さない）
df_w["week_of_year"] = df_w["iso_week"]

# 確認：iso_weekの分布（53があってもOK）
print(df_w["iso_week"].value_counts().sort_index())


In [ ]:
#04
# 季節性（sin / cos）
# 52週周期
df_w["sin_1"] = np.sin(2 * np.pi * df_w["t"] / 52)
df_w["cos_1"] = np.cos(2 * np.pi * df_w["t"] / 52)

# 半年周期（補助）
df_w["sin_2"] = np.sin(4 * np.pi * df_w["t"] / 52)
df_w["cos_2"] = np.cos(4 * np.pi * df_w["t"] / 52)


In [ ]:
#05
# 最終チェック
print(
    "iso_week:", df_w["iso_week"].min(), "→", df_w["iso_week"].max(),
    "| fy_year:", df_w["fy_year"].min(), "→", df_w["fy_year"].max(),
    "| fy_week:", df_w["fy_week"].min(), "→", df_w["fy_week"].max(),
)

display(
    df_w[["Week", "week_start", "t", "iso_year", "iso_week", "fy_year", "fy_week"]].head()
)

In [ ]:
# 06
# 保存（契約成果物のみ）
save_pickle(df_w, Path(OUT_DIR) / "df_w_feat.pkl")

print("Saved:", Path(OUT_DIR) / "df_w_feat.pkl")
